In [1]:
import datetime
import pandas as pd
import numpy as np

# Define the directories that should be used

In [2]:
# Source code to use to run model
my_sourcecode="/glade/u/home/czarakas/cesm_source/cesm_coupled_PPEn11"

# Where you want to put the new folder for the case (where you change things like the 
# namelist, how long to run it, then do case.build, case.submit etc.)
# --- AMY change this
my_casedir="/glade/u/home/czarakas/cesm_cases/coupled_PPE"

# Where the output of the runs should be archived
# probably /glade/scratch/USERNAME/archive/
# --- AMY change this
my_archivedir = "/glade/scratch/czarakas/archive/"

# Where to put the run folder (where log files will go, etc.)
# probably /glade/scratch/USERNAME
# --- AMY change this
my_rundir = "/glade/scratch/czarakas"

# Where you want to save the runscripts that this script will generate
# this can be anything, just make sure the folder exists!
# --- AMY change this
runscript_path = "/glade/u/home/czarakas/coupled_PPE/code/run_scripts/"

# Where to find ensemble member parameter files
# (don't change this unless Daniel changes where these are)
paramfile_path = "/glade/scratch/djk2120/PPEn11/paramfiles/"

# Where to find ensemble member namelist files
#(don't change this unless Daniel changes where these are)
namelistmod_path = "/glade/scratch/djk2120/PPEn11/namelist_mods/"

# The template to use to make the runscripts
filein='run_script_TEMPLATE_Offline.sh'

# Read in crosswalk between offline PPE and coupled PPE

In [3]:
crosswalk = pd.read_csv('CLM5PPE_coupledPPE_crosswalk.csv')

In [4]:
if not ((np.size(crosswalk.key_coupledPPE)) == (np.size(np.unique(crosswalk.key_coupledPPE)))):
    print("Repeated coupled PPE key!")
if not ((np.size(crosswalk.key_CLM5PPE)) == (np.size(np.unique(crosswalk.key_CLM5PPE)))):
    print("Repeated CLM5PPE key!")

In [5]:
# This crosswalk should be in a csv

only_default = False
offline = True

keys_CLM5PPE = crosswalk.key_CLM5PPE
keys_coupledPPE = crosswalk.key_coupledPPE
keys_offlinePPE = crosswalk.key_landonlyPPE

if only_default:
    keys_CLM5PPE=[keys_CLM5PPE.values[0]]
    if offline:
        keys_offlinePPE=[keys_offlinePPE.values[0]]
    else:
        keys_coupledPPE=[keys_coupledPPE.values[0]]

In [6]:
datetag = datetime.datetime.now().strftime("%d%b%Y")
add_datetag_suffix=True

# Generate a bunch of run scripts

In [8]:
key_coupledPPE

'OFFL0012'

In [7]:
for i, key_CLM5PPE in enumerate(keys_CLM5PPE):
    if offline:
        key_coupledPPE=keys_offlinePPE[i]
    else:
        key_coupledPPE=keys_coupledPPE[i]
    
    fileout=runscript_path+'run_script_'+key_coupledPPE+'_PI'
    my_casename=key_coupledPPE+'_PI_v02'
    if add_datetag_suffix:
        fileout=fileout+'_'+datetag+'.sh'
    else:
        fileout=fileout+'.sh'
    
    this_paramfile= '"'+paramfile_path+key_CLM5PPE+'.nc"'
    
    this_namelist= '"'+namelistmod_path+key_CLM5PPE+'.txt"'
    
    f = open(filein,'r')
    filedata = f.read()
    f.close()

    newdata = filedata.replace("TEMPLATE_CASENAME",my_casename)
    newdata = newdata.replace("TEMPLATE_SOURCECODE",my_sourcecode)
    newdata = newdata.replace("TEMPLATE_CASEDIR",my_casedir)
    newdata = newdata.replace("TEMPLATE_ARCHDIR",my_archivedir)
    newdata = newdata.replace("TEMPLATE_RUNDIR",my_rundir)
    newdata = newdata.replace("TEMPLATE_PARAMFILE",this_paramfile)
    newdata = newdata.replace("TEMPLATE_NAMELIST",this_namelist)
    newdata = newdata.replace("TEMPLATE_FILENAME",fileout)

    f = open(fileout,'w')
    f.write(newdata)
    f.close()

***NOTE!*** After running this it will be necessary to do the following in terminal so that you can execute these scripts
> chmod +x FILENAME.sh

After that, you can run them by doing
> ./FILENAME.sh

But stick around to make sure it all works (e.g. that it builds, etc.)